In [5]:
"""
Объединение данных из S3 и Postgres для анализа активных клиентов
Синтетические данные для демонстрации
"""

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, count, mean, max, min, sum as spark_sum, countDistinct
from pyspark.sql.types import StructType, StructField, StringType, DateType, DoubleType, IntegerType
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Конфигурация
ORDERS_S3_PATH = "s3a://startde-project/roma_mesh/orders_filtered"
PG_HOST = "my-postgres-host"
PG_DATABASE = "my-database"
PG_USER = "my-username"
PG_PASSWORD = "my-password"

def generate_synthetic_data(spark): # Генерация синтетических данных
    
    # Схемы данных
    orders_schema = StructType([
        StructField("order_id", StringType(), True),
        StructField("customer_id", StringType(), True),
        StructField("order_purchase_timestamp", DateType(), True),
        StructField("order_status", StringType(), True),
        StructField("order_value", DoubleType(), True)
    ])
    
    customers_schema = StructType([
        StructField("customer_id", StringType(), True),
        StructField("order_count", IntegerType(), True),
        StructField("total_spent", DoubleType(), True),
        StructField("customer_segment", StringType(), True)
    ])
    
    # Генерация Pandas DataFrame
    np.random.seed(42)
    n_orders = 1000
    dates = [datetime(2023, 1, 1) + timedelta(days=x) for x in range(365)]
    
    orders_pd = pd.DataFrame({
        'order_id': [f'ORD_{i:06d}' for i in range(1, n_orders + 1)],
        'customer_id': [f'CUST_{np.random.randint(1, 201):04d}' for _ in range(n_orders)],
        'order_purchase_timestamp': np.random.choice(dates, n_orders),
        'order_status': np.random.choice(['delivered', 'processing', 'shipped', 'canceled'], 
                                      n_orders, p=[0.75, 0.1, 0.1, 0.05]),
        'order_value': np.random.normal(150, 50, n_orders).clip(20, 500)
    })
    
    # Активные клиенты
    customer_stats = orders_pd.groupby('customer_id').agg({
        'order_id': 'count',
        'order_value': 'sum'
    }).reset_index()
    
    customer_stats['order_count'] = customer_stats['order_id'].astype(int)
    customer_stats['total_spent'] = customer_stats['order_value'].astype(float)
    
    customer_stats['customer_segment'] = pd.cut(customer_stats['order_count'], 
                                              bins=[0, 2, 5, 100], 
                                              labels=['Low', 'Medium', 'High'])
    
    customers_pd = customer_stats[['customer_id', 'order_count', 'total_spent', 'customer_segment']].copy()
    
    # Конвертация в Spark DataFrame
    orders_df = spark.createDataFrame(orders_pd, schema=orders_schema)
    customers_df = spark.createDataFrame(customers_pd, schema=customers_schema)
    
    return orders_df, customers_df

def main():
    # Инициализация Spark
    spark = SparkSession.builder \
        .appName("ActiveCustomersAnalysis") \
        .config("spark.hadoop.fs.s3a.access.key", "${S3_ACCESS_KEY}") \
        .config("spark.hadoop.fs.s3a.secret.key", "${S3_SECRET_KEY}") \
        .getOrCreate()
    
    # 1. Чтение данных из S3
    print("=== ШАГ 1: ЧТЕНИЕ ДАННЫХ ИЗ S3 ===")
    print(f"📁 Путь: {ORDERS_S3_PATH}")
    
    # В реальном коде:orders_df = spark.read.parquet(ORDERS_S3_PATH)
    orders_df, active_customers_df = generate_synthetic_data(spark)
    
    # Статистика заказов
    orders_stats = orders_df.groupBy("order_status").agg(count("order_id").alias("count"))
    print("\n📈 Статистика по статусам заказов:")
    orders_stats.show()
    
    value_stats = orders_df.agg(
        mean("order_value").alias("avg_value"),
        max("order_value").alias("max_value"),
        min("order_value").alias("min_value")
    )
    print("💰 Статистика по стоимости заказов:")
    value_stats.show()
    
    print("\nПример данных заказов:")
    orders_df.show(5)
    print("─" * 80)

    # 2. Чтение данных из Postgres
    print("\n=== ШАГ 2: ЧТЕНИЕ ДАННЫХ ИЗ POSTGRES ===")
    print(f"📊 Таблица: orders_filtered")
    print(f"🔗 Подключение: {PG_HOST}:5432/{PG_DATABASE}")
    
    # В реальном коде:
    # postgres_url = f"jdbc:postgresql://{PG_HOST}:5432/{PG_DATABASE}"
    # connection_properties = {"user": PG_USER, "password": PG_PASSWORD, "driver": "org.postgresql.Driver"}
    # active_customers_df = spark.read.jdbc(url=postgres_url, table="orders_filtered", properties=connection_properties)
     
    # Статистика активных клиентов
    segment_stats = active_customers_df.groupBy("customer_segment").agg(count("customer_id").alias("count"))
    print("\n📊 Сегментация клиентов:")
    segment_stats.show()
    
    customer_stats = active_customers_df.agg(
        mean("order_count").alias("avg_orders"),
        max("order_count").alias("max_orders"),
        mean("total_spent").alias("avg_spent")
    )
    print("📈 Статистика активных клиентов:")
    customer_stats.show()
    
    print("\nПример данных активных клиентов:")
    active_customers_df.show(5)
    print("─" * 80)

    # 3. JOIN операция
    print("\n=== ШАГ 3: JOIN ОПЕРАЦИЯ ===")
    print("🔗 Объединение заказов с активными клиентами...")
    
    # Выполняем JOIN
    joined_df = orders_df.alias("orders").join(
        active_customers_df.alias("customers"), 
        col("orders.customer_id") == col("customers.customer_id")
    )
    
    # Статистика после JOIN
    joined_segment_stats = joined_df.groupBy("customers.customer_segment").agg(count("orders.order_id").alias("order_count"))
    print("\n📊 Распределение заказов по сегментам клиентов:")
    joined_segment_stats.show()
    
    print("\nПример объединенных данных:")
    joined_df.select(
        col("orders.order_id"), 
        col("orders.customer_id"), 
        col("orders.order_status"), 
        col("customers.order_count"), 
        col("customers.customer_segment")
    ).show(5)
    print("─" * 80)

    # 4. Подготовка результата
    print("\n=== ШАГ 4: ПОДГОТОВКА РЕЗУЛЬТАТА ===")
    
    # Финальный результат
    result_df = joined_df.select(
        col("orders.order_id"),
        to_date(col("orders.order_purchase_timestamp")).alias("order_date"),
        col("orders.customer_id"),
        col("orders.order_status"),
        col("orders.order_value"),
        col("customers.order_count"),
        col("customers.total_spent"),
        col("customers.customer_segment")
    )
    
    # Финальная статистика
    final_stats = result_df.agg(
        count("order_id").alias("total_orders"),
        countDistinct("customer_id").alias("unique_customers"),
        spark_sum("order_value").alias("total_revenue"),
        mean("order_value").alias("avg_order_value")
    )
    
    print("\n📊 ФИНАЛЬНАЯ СТАТИСТИКА:")
    final_stats.show()
    
    status_final_stats = result_df.groupBy("order_status").agg(count("order_id").alias("count"))
    print("📈 Статусы заказов в финальном наборе:")
    status_final_stats.show()
    
    print("\nПример финальных данных:")
    result_df.show(5)
    print("─" * 80)

    # 5. Запись в Postgres
    print("\n=== ШАГ 5: ЗАПИСЬ РЕЗУЛЬТАТОВ В POSTGRES ===")
    print(f"💾 Таблица: task_4_result_table")
    
    # В реальном коде:
    # postgres_url = f"jdbc:postgresql://{PG_HOST}:5432/{PG_DATABASE}"
    # connection_properties = {"user": PG_USER, "password": PG_PASSWORD, "driver": "org.postgresql.Driver"}
    # result_df.write.mode("overwrite").jdbc(url=postgres_url, table="task_4_result_table", properties=connection_properties)
    
    print("✅ Данные успешно записаны")
    print(f"📊 Записано записей: {result_df.count():,}")
    
    # Итоговая сводка
    print("\n" + "="*50)
    print("🎯 ИТОГИ ETL-ПРОЦЕССА")
    print("="*50)
    
    # Собираем итоговые метрики
    total_orders = orders_df.count()
    active_customers_count = active_customers_df.count()
    joined_count = joined_df.count()
    final_count = result_df.count()
    total_revenue = result_df.agg(spark_sum("order_value")).collect()[0][0]
    efficiency = joined_count/total_orders*100
    
    summary_data = [
        ("Источник S3", f"{total_orders:,} заказов"),
        ("Активные клиенты", f"{active_customers_count:,} клиентов"),
        ("После JOIN", f"{joined_count:,} записей"),
        ("Финальный результат", f"{final_count:,} строк"),
        ("Общая выручка", f"${total_revenue:,.0f}"),
        ("Эффективность", f"{efficiency:.1f}%")
    ]
    
    for metric, value in summary_data:
        print(f"• {metric}: {value}")

    spark.stop()

if __name__ == "__main__":
    main()

=== ШАГ 1: ЧТЕНИЕ ДАННЫХ ИЗ S3 ===
📁 Путь: s3a://startde-project/roma_mesh/orders_filtered

📈 Статистика по статусам заказов:


/opt/conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+------------+-----+
|order_status|count|
+------------+-----+
|     shipped|   91|
|    canceled|   53|
|   delivered|  755|
|  processing|  101|
+------------+-----+

💰 Статистика по стоимости заказов:
+------------------+-----------------+---------+
|         avg_value|        max_value|min_value|
+------------------+-----------------+---------+
|151.16567101617323|306.8874266829997|     20.0|
+------------------+-----------------+---------+


Пример данных заказов:
+----------+-----------+------------------------+------------+------------------+
|  order_id|customer_id|order_purchase_timestamp|order_status|       order_value|
+----------+-----------+------------------------+------------+------------------+
|ORD_000001|  CUST_0103|              2023-08-24|     shipped|148.33850046777695|
|ORD_000002|  CUST_0180|              2023-03-24|   delivered|  139.594157136373|
|ORD_000003|  CUST_0093|              2023-02-11|   delivered|143.57312442805346|
|ORD_000004|  CUST_0015|          